# 📱 1. Scraping Review dari Play Store

Notebook ini akan melakukan scraping review dari aplikasi transportasi online di Google Play Store.

**Aplikasi Target:**
- Gojek
- Grab

**Output:** File CSV di folder `data/raw/`

In [16]:
# Import libraries
import pandas as pd
import time
from google_play_scraper import Sort, reviews
from pathlib import Path
from tqdm import tqdm
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Buat folder jika belum ada
Path('data/raw').mkdir(parents=True, exist_ok=True)

print("✅ Setup complete!")

✅ Setup complete!


## Konfigurasi Scraping

Edit cell di bawah untuk mengatur:
- Package name aplikasi
- Jumlah review yang akan di-scrape
- Bahasa dan negara

In [17]:
# Konfigurasi
APPS = {
    'gojek': 'com.gojek.app',
    'grab': 'com.grabtaxi.passenger'
}

# Jumlah review per aplikasi
COUNT_PER_APP = int(os.getenv('SCRAPE_COUNT_PER_APP', 2000))

# Bahasa dan negara
LANG = 'id'
COUNTRY = 'id'

print(f"📱 Apps: {list(APPS.keys())}")
print(f"📊 Reviews per app: {COUNT_PER_APP}")
print(f"🌍 Language: {LANG}, Country: {COUNTRY}")

📱 Apps: ['gojek', 'grab']
📊 Reviews per app: 2000
🌍 Language: id, Country: id


## Fungsi Scraping

Fungsi untuk scraping review dengan error handling dan progress bar.

In [18]:
def scrape_app_reviews(app_name, package_name, count=2000, lang='id', country='id'):
    """
    Scrape reviews dari Google Play Store.
    
    Args:
        app_name: Nama aplikasi (untuk penamaan file)
        package_name: Package name aplikasi di Play Store
        count: Jumlah review yang akan di-scrape
        lang: Kode bahasa (default: 'id')
        country: Kode negara (default: 'id')
    
    Returns:
        DataFrame berisi review
    """
    print(f"\n🔍 Scraping {app_name}...")
    
    try:
        # Scraping reviews
        result, continuation_token = reviews(
            package_name,
            lang=lang,
            country=country,
            sort=Sort.NEWEST,
            count=count
        )
        
        if not result:
            print(f"⚠️ Tidak ada review ditemukan untuk {app_name}")
            return pd.DataFrame()
        
        # Convert ke DataFrame
        df = pd.DataFrame(result)
        
        # Pilih kolom yang diperlukan
        df = df[['content', 'score', 'at', 'thumbsUpCount']].copy()
        df['app'] = app_name
        
        # Reorder columns
        df = df[['app', 'content', 'score', 'at', 'thumbsUpCount']]
        
        # Simpan ke CSV
        output_file = f'data/raw/{app_name}_reviews.csv'
        df.to_csv(output_file, index=False, encoding='utf-8')
        
        print(f"✅ Scraped {len(df)} reviews")
        print(f"💾 Saved to: {output_file}")
        
        return df
        
    except Exception as e:
        print(f"❌ Error scraping {app_name}: {e}")
        return pd.DataFrame()

## Jalankan Scraping

Proses scraping akan dimulai. **Estimasi waktu: 5-10 menit** (tergantung koneksi internet).

In [19]:
# Scraping semua aplikasi
all_reviews = []

print("=" * 60)
print("🚀 MULAI SCRAPING")
print("=" * 60)

for app_name, package_name in APPS.items():
    df = scrape_app_reviews(
        app_name=app_name,
        package_name=package_name,
        count=COUNT_PER_APP,
        lang=LANG,
        country=COUNTRY
    )
    
    if not df.empty:
        all_reviews.append(df)
    
    # Delay untuk menghindari rate limit
    time.sleep(2)

print("\n" + "=" * 60)
print("✅ SCRAPING SELESAI!")
print("=" * 60)

if all_reviews:
    total_reviews = sum(len(df) for df in all_reviews)
    print(f"\n📊 Total reviews: {total_reviews}")
    print(f"📁 Files tersimpan di: data/raw/")
else:
    print("\n⚠️ Tidak ada review yang berhasil di-scrape")

🚀 MULAI SCRAPING

🔍 Scraping gojek...
✅ Scraped 2000 reviews
💾 Saved to: data/raw/gojek_reviews.csv
✅ Scraped 2000 reviews
💾 Saved to: data/raw/gojek_reviews.csv

🔍 Scraping grab...

🔍 Scraping grab...
✅ Scraped 2000 reviews
💾 Saved to: data/raw/grab_reviews.csv
✅ Scraped 2000 reviews
💾 Saved to: data/raw/grab_reviews.csv

✅ SCRAPING SELESAI!

📊 Total reviews: 4000
📁 Files tersimpan di: data/raw/

✅ SCRAPING SELESAI!

📊 Total reviews: 4000
📁 Files tersimpan di: data/raw/


## Preview Data

Mari kita lihat contoh data yang berhasil di-scrape.

In [22]:
# Load salah satu file untuk preview
if all_reviews:
    print("📋 Preview data Gojek:")
    print("-" * 60)
    
    # Tampilkan 5 review pertama
    df_preview = all_reviews[0].head()
    print(df_preview)
    
    # Statistik
    print("\n📊 Statistik:")
    for df in all_reviews:
        app_name = df['app'].iloc[0]
        print(f"\n{app_name}:")
        print(f"  - Total reviews: {len(df)}")
        print(f"  - Rating distribution:")
        print(df['score'].value_counts().sort_index().to_string())

📋 Preview data Gojek:
------------------------------------------------------------
     app                                            content  score  \
0  gojek  hati hati aplikasi ini sering error , jgn mau ...      1   
1  gojek                                    sangat membantu      5   
2  gojek  sangat membantu keperluan masyarakat untuk har...      5   
3  gojek  transfer ke bank lain memakai aplikasi ini tap...      1   
4  gojek                                  apk yang, bagusss      5   

                   at  thumbsUpCount  
0 2025-10-15 19:02:10              0  
1 2025-10-15 19:01:39              0  
2 2025-10-15 18:43:17              0  
3 2025-10-15 18:37:34              0  
4 2025-10-15 18:30:43              0  

📊 Statistik:

gojek:
  - Total reviews: 2000
  - Rating distribution:
score
1     495
2      81
3      84
4      88
5    1252

grab:
  - Total reviews: 2000
  - Rating distribution:
score
1     301
2      55
3      70
4      95
5    1479


## 🎉 Selesai!

Data review berhasil di-scrape dan disimpan di folder `data/raw/`.

**Next steps:**
- Jalankan notebook `2_preprocessing.ipynb` untuk membersihkan data
- File yang dihasilkan:
  - `data/raw/gojek_reviews.csv`
  - `data/raw/grab_reviews.csv`

**Catatan:**
- Jika terjadi error, coba kurangi `COUNT_PER_APP`
- Scraping mematuhi rate limit untuk menghindari IP ban
- Data adalah review publik dari Play Store
- Default: 2000 reviews per aplikasi (total ~4000 reviews)